<a href="https://colab.research.google.com/github/lucarinelli/conditional_text_generation/blob/move-to-python-files/gpt2_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GPT2 experiments of Conditional Text Generation

Add a description here...

#Setup

SSH for developing and debugging purposes, useful to quickly explore all the files involved in the repo and add or fix things here and there. 

Do not enable if you are just running the experiment.

In [ ]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared
# Comment or un-comment the next line to disable/enable ssh
#launch_ssh_cloudflared(password="2N6.ufRjL,Zp:GfcJuh?TQ")

Mount goole drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Check allocated GPU, hope for something better than a K80...

In [ ]:
!nvidia-smi

Install the needed python packages

In [ ]:
!pip install --quiet transformers datasets tokenizers sacrebleu wandb

Clone our repository to get our utilities and overrides

In [ ]:
!git clone https://github.com/lucarinelli/conditional_text_generation.git /content/conditional_text_generation

Add our repository `src` folder to python path

In [ ]:
import sys
import os

module_path = os.path.abspath("/content/conditional_text_generation/src")
if module_path not in sys.path:
    sys.path.append(module_path)

Setup and connect to Weights and Biases to store logs and results

**REMEMBER TO SET WANDB_PROJECT TO THE CORRECT VALUE**

In [ ]:
import wandb

%env WANDB_PROJECT=ctrl_dry_runs
%env WANDB_ENTITY=polito_aiml2021_textgen
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
%env WANDB_SILENT=true

wandb.login()

Set training arguments and other experiment parameters

In [ ]:
from experiment_parameters import ExperimentParameters
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./data/results",  # output directory
    save_total_limit=3,
    num_train_epochs=3,  # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=1,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,
    logging_dir='./data/logs',  # directory for storing logs
    evaluation_strategy="epoch",
    report_to="wandb",
    load_best_model_at_end=True,
    remove_unused_columns=False
)

experiment_parameters = ExperimentParameters(training_args=training_args)

#Datase
We download and load the COCO captions dataset.

We join in a single item the caption for an image with the categories and/or supercategories associated to objects present in the image.
Categories and/or supercategories are used as control codes depending on the experiment settings.

The dataset is post processed to train the model with different combinations of control codes for each caption, depending on the experiment parameters. The output of the postprocessing is saved on .json files that are then loaded and further handled by the Dataset class provided by HuggingFace datasets (used for its performance and caching abilities).

In [ ]:
from captions_dataset import load_our_dataset

dataset_train, dataset_val = load_our_dataset(experiment_parameters, data_path="/content/data")

#Tokenization

In [ ]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained(experiment_parameters['model'])
tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer before added special tokens "+str(len(tokenizer)))

if experiment_parameters["use_control_codes"] and experiment_parameters["control_codes_type"] == "special_token":
    special_tokens_dict = {'additional_special_tokens': control_codes}
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    print("added "+str(num_added_toks)+" tokens to the pretrained tokenizer")

In [ ]:
def encode(examples):
    encoded = tokenizer(examples['caption'], truncation=True, max_length=64, padding="max_length")
    encoded['labels'] = encoded['input_ids']
    encoded['image_id'] = examples['image_id']
    return encoded

dataset_train_encoded = dataset_train.map(encode, batched=True)
dataset_val_encoded = dataset_val.map(encode, batched=True)

#Model

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained(experiment_parameters['model'], pad_token_id=tokenizer.eos_token_id)
model.resize_token_embeddings(len(tokenizer))

#Metrics

In [ ]:
import datasets

def compute_metrics(pred, image_ids):
    preds = pred.predictions
    metric = datasets.load_metric('sacrebleu')

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    references_local_list = [references[image_id.item()] for image_id in image_ids]

    final_score = metric.compute(predictions=preds, references=references_local_list)
    
    return {
        'bleu': final_score
    }

#Training

In [ ]:
import random
import torch
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
from our_trainer import OurTrainer

dataset_train_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
dataset_val_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'image_id'])

trainer = OurTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset_train_encoded,         # training dataset
    eval_dataset=dataset_val_encoded,
    compute_metrics=compute_metrics,
    )

In [ ]:
trainer.train()

config = wandb.config
config.update(experiment_parameters)

In [ ]:
trainer.save_model("./data/results")
wandb.finish()